![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv('bank_marketing.csv')
df.info()
df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
5,5,45,services,married,basic.9y,unknown,no,may,5,198,1,0,nonexistent,93.994,4.857,no
6,6,59,admin.,married,professional.course,no,no,may,3,139,1,0,nonexistent,93.994,4.857,no
7,7,41,blue-collar,married,unknown,unknown,no,may,12,217,1,0,nonexistent,93.994,4.857,no
8,8,24,technician,single,professional.course,no,yes,may,21,380,1,0,nonexistent,93.994,4.857,no
9,9,25,services,single,high.school,no,yes,may,5,50,1,0,nonexistent,93.994,4.857,no


In [2]:
# Client table
# Create the client dataframe by assigning appropriate columns
client = df[['client_id','age', 'job', 'marital', 'education','credit_default','mortgage']]

# Scrub the job column in the dataframe by replacing characters
client['job'] = client['job'].str.replace('.','_')

# Scrub the education column in the dataframe by replacing incorrect characters
# Replace records with 'unknown' values in the education column with 'np.NaN' to signify missing data
client['education'] = client['education'].str.replace('.','_')
client['education'] = client['education'].str.replace('unknown','np.NaN')

# Correct dtype of the credit default and mortgage columns
client[['credit_default','mortgage']] = client[['credit_default','mortgage']].astype(bool)
print(client)

       client_id  age  ... credit_default mortgage
0              0   56  ...           True     True
1              1   57  ...           True     True
2              2   37  ...           True     True
3              3   40  ...           True     True
4              4   56  ...           True     True
...          ...  ...  ...            ...      ...
41183      41183   73  ...           True     True
41184      41184   46  ...           True     True
41185      41185   56  ...           True     True
41186      41186   44  ...           True     True
41187      41187   74  ...           True     True

[41188 rows x 7 columns]


In [3]:
# Campaign table
# Create the campaign dataframe by assigning appropriate columns
campaign = df[['client_id', 'number_contacts', 'contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','day','month']]

# Add column year with value '2022' for all records to the campaign df
campaign['year']='2022'

# Convert day column to str instead of integer
campaign['day']= campaign['day'].astype(str)

# Create last_contact_date column "YYYY-MM-DD"
campaign['last_contact_date'] = campaign['year'] + "-" + campaign['month'] + "-" + campaign['day']

# Drop the day, year, and month columns since it's not needed in the df after creating the last_contact_date column
campaign = campaign.drop(columns=['day','year','month'])

# Convert last_contact_date column to date dtype
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")
# Map campaign data to the previous_outcome column
campaign ['previous_outcome'] = campaign['previous_outcome'].map({'success':1,
                                'failure':0,
                            'nonexistent':0})
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes':1,'no':0})
# Convert previous_outcome and campaign_outcome columns to boolean dtype
campaign[['previous_outcome','campaign_outcome']] = campaign[['previous_outcome','campaign_outcome']].astype(bool)
print(campaign.info())
print(campaign.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB
None
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1

In [4]:
economics = df[['client_id','cons_price_idx','euribor_three_months']]
print(economics.info())
print(economics.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None
   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


In [5]:
# Export cleaned client, campaign, and economics tables to csv files using pandas to_csv() function

client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)